<a href="https://colab.research.google.com/github/andresni/miniPHI/blob/master/miniPHI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Approximations and heuristics of PHI
This is a notebook for generating logic gate networks, calculating PHI on them, as well as approximations and heuristics of PHI.

PHI, $\theta$, is a measure of integrated information, from the Integrated Information Theory of Consciousness (IIT) by Guilio Tononi (Oizumi et al. 2014), and reflects a system's (here a logic gate network) cause effect power over itself. In other words, $\theta$ reflects how much a network in a given state causally restricts its past and future state space, above and beyond how much its parts causally restricts their past and future state spaces.

The issue with $\theta$ as currently formulated is that it's not feasible to calculate $\theta$ for any system with more than a dozen nodes. Thus, it is of interest to either find heuristics that capture similar underlying aspects as $\theta$, and approximations to $\theta$ that reduce some of the computational complexity of the original calculations.

In this notebook, we will provide code that generates networks and for each network automatically calculates $\theta$ as well as a selection of other properties.

See the paper by [Nilsen, Juel, and Marshall (2019)](https://www.mdpi.com/1099-4300/21/5/525) for more information.


### Installs and imports

In [1]:
%pip install pyphi
%pip install git+https://github.com/andresni/pyconscious.git

import numpy as np
import pandas as pd
import copy
import time
import itertools
import os
import csv
import random
import scipy.io as sio
import scipy as spi
import pyphi
import pyconscious as pc

  Cloning https://github.com/andresni/pyconscious.git to /tmp/pip-req-build-o5g0a4jm
  Running command git clone -q https://github.com/andresni/pyconscious.git /tmp/pip-req-build-o5g0a4jm
  Created wheel for pyconscious: filename=pyconscious-0.1-cp36-none-any.whl size=7986 sha256=96ae16b4fc04eac2d95c57f0ac3072533931ac14f19efed3b8a3b1f24e626cce
  Stored in directory: /tmp/pip-ephem-wheel-cache-rogntt2h/wheels/23/ad/ac/47ef26f546b837efed439d7099e44a822bde018abce384d259
Successfully built pyconscious

Welcome to PyPhi!

If you use PyPhi in your research, please cite the paper:

  Mayner WGP, Marshall W, Albantakis L, Findlay G, Marchman R, Tononi G.
  (2018). PyPhi: A toolbox for integrated information theory.
  PLOS Computational Biology 14(7): e1006343.
  https://doi.org/10.1371/journal.pcbi.1006343

Documentation is available online (or with the built-in `help()` function):
  https://pyphi.readthedocs.io

To report issues, please use the issue tracker on the GitHub repository:
  https:

### Functions for generating networks
Here we generate networks $M$ with the property

$$M_{i,j} =
 \begin{pmatrix}
  0 & w_{1,2} & \cdots & w_{1,j} \\
  w_{2,1} & 0 & \cdots & w_{2,j} \\
  \vdots  & \vdots  & \ddots & \vdots  \\
  w_{i,1} & w_{i,2} & \cdots & 0
 \end{pmatrix}$$
 
 where $w_{i, j}=[-1,0,1]$ is the weight between nodes $i$ and $j$.

In [0]:
def net_gen_next(Params):
    networks = []
            
    for size in range(Params["minsize"], Params["maxsize"] + 1):
        temp_net = []
        for x in range(0, Params["nrnetworks"]):
            temp = np.zeros([size,size])
            
            conn_prob = np.random.uniform(0.2, 1.0)
            inh_prob = np.random.uniform(0.0, 0.8)
            
            for xx in range(0,size):
                for yy in range(0,size):
                    if not xx==yy:
                        if np.random.rand() < conn_prob:
                            temp[xx][yy] = np.random.normal(1, 0)
                            if np.random.rand() < inh_prob:
                                temp[xx][yy] = np.random.normal(-1, 0)
            
            temp2 = list(itertools.permutations([it for it in range(size)]))
            temp3 = [temp[:, list(it)][list(it)] for it in temp2]
            
            yes = 0
            for ii in temp3:
                for zz in temp_net:    
                    if len(np.unique([ii, zz], axis=0)) < 2:
                        yes = 1
                        break
            if not yes:
              temp_net.append(copy.copy(temp))
                    
        for xx in temp_net:
            networks.append(xx)
    
    return np.array(networks)

### Functions for generating activity
Here are functions that generate activity $A_t$, where 

$$A_t = \left[\begin{array}{c}a^{t-1}_1\\a^{t-1}_2\\\vdots\\a^{t-1}_i\end{array}\right]\cdot M \cdot C=\left[\begin{array}{c}a^t_1\\a^t_2\\\vdots\\a^t_i\end{array}\right]$$

with $a^t_i=[0,1]$, $t$ is timestep, $i$ is network element, and $A_0$ is set to one of $2^i$ possible binary states. $a^t_i=1$ if $a^t_i\geq 1$ and $0$ otherwise. $C$ is a noise parameter (for connections only).

### Noise
There are three kinds of possible noise.

1. Connection noise

$C$ is an array of length $i$ with random values from a gaussian distribution with $\mu=0, \sigma=s$. 

2. Node noise

$a^t_i=1$ has a probability of flipping to $a^t_i=0$ with $P=p$, and the opposite with equal probability.

3. Embedded noise

The network $M$ is embedded in a much larger network that is "unknowkn" to us. I.e. $M_{small}$ is a subset of $M_{large}$.

In [0]:
def createLOLI(nodes): # Creates an ordered list that iterates through all possible permutations of a binary array.
    LOLI = np.zeros((int(2**nodes), int(nodes)))
    i = 1
    for z in range(0, len(LOLI[0, :])):
        for x in range(i, len(LOLI[:, 0])):
            if LOLI[x - i, z] == 1:
                LOLI[x, z] = 0
            else:
                LOLI[x, z] = 1
        i = i * 2       
    return LOLI

def makeStateByNode(spm,LOLI): # Convert activity to state by node probability matrix

    sbn=np.zeros((len(LOLI),len(LOLI[0])+1))
    a = int(len(spm[0][0]) - len(LOLI[0]))
    
    for x in range(len(spm)):
      for i in range(len(spm[x])-1):
        b=spm[x][i][:-a] if a > 0 else spm[x][i]
        c=spm[x][i+1][:-a] if a > 0 else spm[x][i+1]
        indexa=LOLI.index(list(b))
        c=np.append(c,1)
        sbn[indexa]=np.add(sbn[indexa],c)
        
    for i in range(0,len(sbn)):
        for q in range(0,len(sbn[i])-1):
            sbn[i,q]=sbn[i,q]/sbn[i][-1] if sbn[i][-1] > 0 else 0
            sbn[i,q]=np.round(sbn[i,q],4)
    
    return sbn[:,0:-1]

def makeStateByState(spm,LOLI): # Convert activity to state by state probability matrix

    sbn=np.zeros((len(LOLI),len(LOLI)))
    a = int(len(spm[0][0]) - len(LOLI[0]))
    
    for x in range(len(spm)):
      for i in range(len(spm[x])-1):  
        b=spm[x][i][:-a] if a > 0 else spm[x][i]
        c=spm[x][i+1][:-a] if a > 0 else spm[x][i+1]
        indexa=LOLI.index(list(b))
        indexb=LOLI.index(list(c))
        sbn[indexa,indexb]+=1
        
    for i in range(0,len(sbn)):
        s=np.sum(sbn[i])
        for q in range(0,len(sbn[i])):
            sbn[i,q]=sbn[i,q]/s if s > 0 else 0
            sbn[i,q]=np.round(sbn[i,q],4)
    
    return sbn

def gen_transition_prob_matrices_state(results): # Create transition probability matrix (state by state)
   
    tpm = []    
    for i in range(len(results["partial_activity"])):
        tpm.append(copy.deepcopy(makeStateByState(results["partial_activity"][i],results["LOLI"][i])))
    return {"tpms":copy.deepcopy(tpm)}

def gen_transition_prob_matrices_node(results): # Create transition probability matrix (state by node)
    
    tpm = []
    for i in range(len(results["partial_activity"])):
        tpm.append(copy.deepcopy(makeStateByNode(results["partial_activity"][i],results["LOLI"][i])))
    return {"tpmn":copy.deepcopy(tpm)}

def activity(results,Params): # Generate activity given a network
    
    all_net_act = []
    all_net_pact = []
    all_net_LOLI = []
    
    for i in results["networks"]:
        partial = []
        LOLI = createLOLI(len(i))

        replace = True if Params["states"] > len(LOLI) else False
        states = np.random.choice(list(range(len(LOLI))), Params["states"], replace = replace)
        states = list(states) * Params["steps"]

        for L in states:
            state_block = [LOLI[L].tolist()]
            for rounds in range(Params["samples"]):
                temp = np.zeros(len(i))
                for x in range(len(i)):
                    if LOLI[L][x] > 0:
                        for y in range(len(i)):
                          noise = np.random.uniform(-Params["noiseLVL"], Params["noiseLVL"]) if Params["noise"] == "con" else 0
                          temp[y] = temp[y] + i[x][y] + np.random.uniform(noise, noise)
                for x in range(len(i)):
                  temp[x] = 1 if temp[x] >= 1 else 0
                  if Params["noise"] == "node":
                    temp[x] = [1, 0][int(temp[x])] if np.random.rand() < Params["noiseLVL"] else [0, 1][int(temp[x])]
                state_block.append(temp.tolist())
            partial.append(copy.deepcopy(state_block))
        
        all_net_pact.append(copy.deepcopy(partial))
        all_net_LOLI.append(LOLI.tolist())
        
    return {"partial_activity": all_net_pact, "LOLI": all_net_LOLI}

def net_gen_noise(net,Params):
  result = []
  if Params["noiseLVL"] > 0:
    conn_net = np.zeros((Params["noiseLVL"]+len(net),Params["noiseLVL"]+len(net)))
    u, c = np.unique(net, return_counts = True)
    nrcons = np.sum(np.abs(net)) / np.sum(net**0)
    nrinh = c[0] / np.sum(np.abs(net)) if -1 in u else 0
  
    for x in range(len(conn_net)):
      for y in range(len(conn_net)):
        conn_net[x,y] = 1 if np.random.rand() < nrcons else 0
        conn_net[x,y] = -1 if np.random.rand() < nrinh and conn_net[x,y] == 1 else conn_net[x,y]
    conn_net[:-Params["noiseLVL"],:-Params["noiseLVL"]] = net
  else:
    conn_net = net
  
  LOLI = statsPlots.createLOLI(len(net))
  
  replace = True if Params["states"] > len(LOLI) else False
  states = np.random.choice(list(range(len(LOLI))),Params["states"],replace=replace)
  states = list(states) * Params["steps"]
  for L in states:
    S=list(LOLI[L])  
    state_block = [list(np.random.randint(0, 2, size=(len(conn_net))))]
    if Params["noiseLVL"] > 0:
      state_block[-1][:-Params["noiseLVL"]] = copy.deepcopy(S)
    else:
      state_block[-1] = copy.deepcopy(S)
    S=copy.deepcopy(state_block[-1])
    for rounds in range(Params["samples"]):
      temp = np.zeros(len(conn_net))
      for x in range(len(conn_net)):
        for y in range(len(conn_net)):
          temp[y]=temp[y]+(S[x]*conn_net[x][y])
      for x in range(len(net)):
          temp[x] = 1 if temp[x] >= 1 else 0
      S = list(temp)
      state_block.append(list(temp))
        
    result.append(copy.deepcopy(state_block))
  return result

### Analysis tools

Now we just need to generate some analysis functions.

**PHI 3.0 calculations**

In [0]:
def phi(struct,net,config_opt):
    
    result = {}

    whole_sia = []
    whole_phi = []
    whole_concepts = []
    statenr = []  
    all_sia =[]
    mc_sia = []
    mc_phi = []
    mc_size = []
    mc_whole = []
    mc_concepts = []
    mc_mc = []
    time = []
  
    pyphi.config.ASSUME_CUTS_CANNOT_CREATE_NEW_CONCEPTS = config_opt["no_new_concepts"]
    pyphi.config.CUT_ONE_APPROXIMATION = config_opt["cut_one_aprox"]
    pyphi.config.MAXIMUM_CACHE_MEMORY_PERCENTAGE = config_opt["max_cache"]
    pyphi.config.CACHE_POTENTIAL_PURVIEWS = config_opt["cache_pot_purw"]
    pyphi.config.CLEAR_SUBSYSTEM_CACHES_AFTER_COMPUTING_SIA = config_opt["clear_sys_cache"]
    pyphi.config.CACHE_SIAS = True
    pyphi.config.CACHE_REPERTOIRES = True
    
    pyphi.config.PARTITION_TYPE = config_opt["part_type"]
    pyphi.config.PROGRESS_BARS=False
    pyphi.config.PARALLEL_CUT_EVALUATION=False
    pyphi.config.PARALLEL_COMPLEX_EVALUATION=True
    pyphi.config.NUMBER_OF_CORES=16
    pyphi.config.LOG_STDOUT_LEVEL="ERROR"
    pyphi.config.LOG_FILE_LEVEL="ERROR"
    pyphi.config.log()

    tpms = struct["tpmn"][net]
    LOLI = struct["LOLI"][net]
    conmat = np.sign(struct["networks"][net])

    labels = [chr(97+i) for i in range(len(conmat))]
    network=pyphi.Network(tpms,cm=np.abs(conmat),node_labels=labels)

    temp = [calcphi(LOLI[s],network,labels,s,len(LOLI)) for s in range(len(LOLI))]
   
    for xs in temp:
        whole_sia.append(copy.deepcopy(xs["whole_sia"]))
        whole_phi.append(copy.deepcopy(xs["whole_phi"]))
        whole_concepts.append(copy.deepcopy(xs["whole_concepts"]))
        statenr.append(copy.deepcopy(xs["statenr"]))
        mc_sia.append(copy.deepcopy(xs["mc_sia"]))
        mc_phi.append(copy.deepcopy(xs["mc_phi"]))
        mc_size.append(copy.deepcopy(xs["mc_size"]))
        mc_whole.append(copy.deepcopy(xs["mc_whole"]))
        mc_concepts.append(copy.deepcopy(xs["mc_concepts"]))
        mc_mc.append(copy.deepcopy(xs["mc_mc"]))
        all_sia.append(copy.deepcopy(xs["all_sia"]))
        time.append(copy.deepcopy(xs["time"]))
    
    return {"mc_phi":mc_phi}

def calcphi(state,network,labels,s,tot):
    result = {"statenr":s,"whole_sia":np.NaN,"whole_phi":np.NaN,"whole_concepts":np.NaN,"all_sia":np.NaN,"time":np.NaN}
    result.update({"mc_sia":np.NaN,"mc_phi":np.NaN,"mc_size":np.NaN,"mc_whole":np.NaN,"mc_concepts":np.NaN,"mc_mc":np.NaN})
    phitime = time.time()
    cstate=np.array(state).astype(int)    

    try:
        whole_sia = pyphi.compute.network.all_complexes(network,cstate)
                      
        for i in range(len(whole_sia)):
            if (whole_sia[i].phi > result["whole_phi"] and len(whole_sia[i].subsystem)== len(labels)) or np.isnan(result["whole_phi"]):
                result["whole_phi"] = whole_sia[i].phi
                result["whole_concepts"] = len(whole_sia[i].ces)
                result["whole_sia"] = np.NaN#whole_sia[i]
            if whole_sia[i].phi > result["mc_phi"] or np.isnan(result["mc_phi"]) or (whole_sia[i].phi == result["mc_phi"] and len(whole_sia[i].ces) > result["mc_size"]):
                result["mc_phi"] = whole_sia[i].phi
                result["mc_concepts"] = len(whole_sia[i].ces)
                result["mc_sia"] = np.NaN#whole_sia[i]
                result["mc_size"] = len(whole_sia[i].subsystem)
                result["mc_mc"] = whole_sia[i].subsystem.node_indices
                result["mc_whole"] = 1 if result["mc_size"] == len(labels) else 0
        result["all_sia"]=len(whole_sia)
        phitime=time.time()-phitime
        result["time"]=copy.deepcopy(phitime)

        return result
    
    except Exception as e:
        if "cannot be reached" in str(e):
            return result
        else:
            print("Failed big-time")
            print(str(e))
            crash=10/0

def complexity(results,n):
  lz = []
  for i in results["partial_activity"][n]:
    b = pc.functions.concatenate(np.array(i).astype(int),concat="space")
    c = pc.functions.compress(b)
    d = pc.functions.compress("".join(random.sample([str(random.randint(0,1)) for x in range(len(b))],len(b))))
    lz.append(c/d)
  return lz


## Ready to rumble

**First we define parameters...**

"*Steps*" which refers to how many states should a network visit following a given initialization. This will be normalized if multiple network sizes are used so that the information content of different network sizes are the same (i.e. fewer nodes = more steps).

"*Samples*" refers to how many times should a given initialization be repeated.

"*States*" refers to how many initializations should be performed

"*noiseLVL*" refers to how much noise should be present in the network activity.

"*noise*" refers to type of noise, with options "con" (fuzzy connections), "node" (fuzzy nodes), "net" (embedded network, i.e. hidden background nodes influencing network).

In [0]:
Params = {"nrnetworks": 100,  # generate how many networks of each size
          "minsize": 3,  # min node count
          "maxsize": 3,  # max node count
          "steps": 50,  # number of steps per network
          "samples": 50,  # number of samples per state
          "states": 20,  # number of states to start in
          "noiseLVL": 0.00,  # plus minus noise
          "noise": "none",  # kind of noise (none, con, node, net)
          }

**Then we generate networks and associated activity**

In [0]:
results = {"networks": net_gen_next(Params)}

results.update(activity(results,Params))
results.update(gen_transition_prob_matrices_state(results))
results.update(gen_transition_prob_matrices_node(results))


**And now we can finally calculate PHI3.0 for these networks**

Be warned though, this code is not optimized for GPUs, supercomputers, etc. Depending on your resources, networks with 6 nodes can take up to an hour, 7 nodes can take up to a day. So don't go running too many big nets. The config parameters instruct how PHI should be calculated (if using approximations for certain steps of the calculation, partitioning type, etc.). Also consider running with local resources.

In [87]:
pf = pd.DataFrame(columns=["Net","Nodes","PhiMax","PhiMean","LZmax","LZmean"])

# Config options - 
config_opt = {"no_new_concepts": False, # No new concepts approximation
              "cut_one_aprox": False, # Cut one approximation
              "part_type": "BI", # Partition type (BI, TRI, FULL) - approximation
              "par_comp_eval": True, # Parallel computation (False if par. comp. higher up)
              "max_cache": 50, # Cache size
              "cache_pot_purw": True, # Faster but higher mem. use
              "clear_sys_cache": False, # Slower but lower mem. use
              }

for i in range(len(results["networks"])):
  a = []
  t = phi(results,i,config_opt)
  a.extend([np.nanmean(t["mc_phi"]),np.nanmax(t["mc_phi"])])
  t = complexity(results,i)
  a.extend([np.nanmean(t),np.nanmax(t)])
  pf = pf.append({"Net":i, "Nodes":len(results["networks"][i]), "PhiMax":a[1], "PhiMean":a[0],"LZmax":a[3], "LZmean":a[2]}, ignore_index=True)
print(pf)

     Net  Nodes    PhiMax   PhiMean     LZmax    LZmean
0    0.0    3.0  0.000000  0.000000  0.488889  0.411004
1    1.0    3.0  0.629633  0.304290  0.600000  0.493162
2    2.0    3.0  0.256945  0.093254  0.600000  0.507084
3    3.0    3.0  0.112144  0.078699  0.577778  0.483834
4    4.0    3.0  0.468339  0.308280  0.600000  0.490305
..   ...    ...       ...       ...       ...       ...
56  56.0    3.0  0.000000  0.000000  0.600000  0.474495
57  57.0    3.0  0.000000  0.000000  0.600000  0.526020
58  58.0    3.0  0.000000  0.000000  0.590909  0.484841
59  59.0    3.0  0.000000  0.000000  0.577778  0.471894
60  60.0    3.0  0.382579  0.261990  0.577778  0.514799

[61 rows x 6 columns]


***
The above is a simple demonstration. To test your own measures, add them as a function. The item "partial_activity" reflects $A_t$ initialized from each possible state (out of $2^i$), and "networks" reflect $M$. As output from the $\theta$ calculations is only the $\theta$ of the maximally irreducible cause-effect structure. However, other alternatives such as $\theta_{whole}$ can be output by changing the function.

For questions or otherwise, please contact **sevenius.nilsen@gmail.com**